In [ ]:
import numpy as np
from tensorflow.keras.layers import Layer
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import Model
class Linear(keras.layers.Layer): #defining linear layer
  """y = w.x + b"""

  def __init__(self, units=3):
      super(Linear, self).__init__()
      self.units = units

  def build(self, input_shape):
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

  def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b

class MyModel(tf.keras.Model): # defining stack of layers 
    def __init__(self):
        super(MyModel, self).__init__()
        self.d0 = Linear(3)
        self.d1 = Linear(4)
        self.d2 = Linear(2)
        

    def call(self, inputs, training=None, **kwargs):
        x = self.d0(inputs)
        x = tf.nn.relu(x)
        x = self.d1(x)
        x = tf.nn.relu(x)
        return self.d2(x)



In [ ]:
n=400 #number of input samples
d=3 # number of dimensions
x=tf.convert_to_tensor(tf.Variable(np.random.rand(400, 3), dtype=tf.float32)) # tensorflow x variable initialization
w_true=tf.constant([[5,1,1],[4,1,1]], dtype=tf.float32) # defining weights for two non linear equations
w_true=tf.transpose(w_true)
bias_true=tf.constant([1], dtype=tf.float32)
y1=tf.matmul(x**2,w_true)+tf.matmul(x,w_true)+bias_true #y_true calculation
model = MyModel()
loss_obj_reg = tf.keras.losses.MeanAbsoluteError()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1)
num_epochs=50
#loss_reg = tf.keras.metrics.Mean(name='regression loss')
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:  # using auto differentiation
      pred_reg = model(x)    
      loss = loss_obj_reg(y1, pred_reg)
        #print(reg_loss)
        #print("Step %d, loss %d" %(epoch, reg_loss))  
      gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(float(loss))


6.538780689239502
6.41911506652832
6.274316310882568
6.090416431427002
5.8565802574157715
5.560863018035889
5.190067291259766
4.730538845062256
4.171006679534912
3.52610445022583
2.8513009548187256
2.258810520172119
1.9409805536270142
2.0017483234405518
2.3864996433258057
2.6691763401031494
2.623897075653076
2.365532636642456
2.084580659866333
1.8909534215927124
1.8171921968460083
1.8352340459823608
1.890499234199524
1.9572815895080566
2.006730079650879
2.02164363861084
1.9997047185897827
1.9435621500015259
1.8622158765792847
1.7694778442382812
1.68231999874115
1.6142914295196533
1.5758099555969238
1.5710303783416748
1.5847525596618652
1.586797833442688
1.5509835481643677
1.4802751541137695
1.3986105918884277
1.330241084098816
1.284846544265747
1.2578758001327515
1.234710454940796
1.1973377466201782
1.1366947889328003
1.0556315183639526
0.9674528241157532
0.8972540497779846
0.8500102758407593
0.807227373123169


In [ ]:
#rawing interactive 3d plots
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [ ]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y1)
y_pred_red = TSNE(n_components=1).fit_transform(model(x))
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')

plot_intereactive_3d(X_red,y_true_red,y_pred_red)

  

X_red: (400, 2), y_true_red: (400, 1), y_pred_red: (400, 1)
